In [3]:
#  Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler


In [13]:
# 📥 Chargement des données pré-nettoyées
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
X = pd.read_csv("../data/X_imputed.csv")
X_test = pd.read_csv("../data/X_test_imputed.csv")
y = pd.read_csv("../data/y.csv").squeeze()  # pour transformer en Series

print("Données chargées :", X.shape, y.shape, X_test.shape)


Données chargées : (3960, 4) (3960,) (20, 4)


In [9]:
# Supprimer les lignes où la cible est NaN
combined = pd.concat([X, y], axis=1)
combined_clean = combined.dropna(subset=["sii"])

# Re-séparer X et y après nettoyage
X = combined_clean.drop(columns=["sii"])
y = combined_clean["sii"]

print(" Données prêtes après nettoyage :")
print("X :", X.shape)
print("y :", y.shape)



 Données prêtes après nettoyage :
X : (2736, 4)
y : (2736,)


In [10]:
# Normalisation (optionnelle mais propre)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split train / validation
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Modèle de base
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_val)

# Évaluation
qwk = cohen_kappa_score(y_val, y_pred, weights='quadratic')
print(f"Quadratic Weighted Kappa (QWK) : {qwk:.4f}")


✅ Quadratic Weighted Kappa (QWK) : 0.3088


## Conclusion – Modèle de base

On vient de construire notre premier modèle de classification simple avec un `RandomForestClassifier`.

- On a utilisé uniquement **4 variables numériques** disponibles dans le train et le test.
- Les valeurs manquantes ont été gérées proprement par imputation.
- Après entraînement et validation, on a obtenu un **score Quadratic Weighted Kappa de 0.3088**.

Ce modèle est notre **baseline** : il est simple mais déjà fonctionnel.  
Il nous servira de point de comparaison pour toutes les futures améliorations.

Prochaine étape : soit faire une première soumission Kaggle, soit améliorer notre modèle avec de nouvelles features ou un algorithme plus puissant.


In [14]:
# Prédiction sur le test set
X_test_scaled = scaler.transform(X_test)
y_test_pred = model.predict(X_test_scaled)

# Création du fichier de soumission
submission = pd.DataFrame({
    "id": test["id"],  
    "sii": y_test_pred
})

# Sauvegarde en CSV
submission.to_csv("../outputs/submission_baseline.csv", index=False)

print(" Fichier de soumission généré !")
submission.head()


✅ Fichier de soumission généré !


,id,sii
0,00008ff9,2.0
1,000fd460,0.0
2,00105258,1.0
3,00115b9f,1.0
4,0016bb22,0.0
